In [ ]:
!git clone https://github.com/rojenarda/CoReEcho
!mv CoReEcho/coreecho coreecho
!mv CoReEcho/requirements.txt requirements.txt
!mv CoReEcho/test_start_indexes.pkl test_start_indexes.pkl
!mv CoReEcho/train_first_stage.py train_first_stage.py
!mv CoReEcho/train_second_stage.py train_second_stage.py
!mv CoReEcho/utils.py utils.py

!rm -rf CoReEcho


In [ ]:
!pip install albumentations==1.2.1
!pip install matplotlib==3.7.3
!pip install numpy==1.22.4
!pip install pandas==1.4.3
!pip install pytorch_lightning==1.9.5
!pip install pyyaml==6.0
!pip install scikit_image==0.19.3
!pip install scikit_learn==1.3.2
!pip install scipy==1.8.1
!pip install torch==1.12.1+cu116 --extra-index-url https://download.pytorch.org/whl/cu116
!pip install torchvision==0.13.1+cu116 --extra-index-url https://download.pytorch.org/whl/cu116
!pip install torchaudio==0.12.1+cu116 --extra-index-url https://download.pytorch.org/whl/cu116
!pip install opencv_contrib_python==4.4.0.46
!pip install seaborn==0.13.0
!pip install timm==0.6.12
!pip install torcheval==0.0.7
!pip install lightning_utilities==0.9.0
!pip install umap_learn==0.5.4
!pip install plotly
!pip install comet_ml

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

In [ ]:
import os

from train_first_stage import main as train_first_stage
from train_second_stage import main as train_second_stage

In [ ]:
ECHO_FILE_ROOT = os.path.join('drive', 'MyDrive', 'echo')
DATA_DIR = os.path.join(ECHO_FILE_ROOT, 'dataset', 'resized') # HFpEF DIR
UNIFORMER_WEIGHTS_DIR = os.path.join(ECHO_FILE_ROOT, 'models', 'uniformer_small_k400_16x8.pth')
COMET_API_KEY = ''
MODEL_PATH = os.path.join(ECHO_FILE_ROOT, 'models', 'coreecho')


# Training

## Stage 1

In [ ]:
stage_1_args = [
    '--data_folder', DATA_DIR,
    '--pretrained_weights', UNIFORMER_WEIGHTS_DIR,
    '--project_name', 'coreecho-classifier-stage-1',
    '--model', 'uniformer_small',
    '--num_workers', '8',
    '--batch_size', '16',
    '--frames', '36',
    '--frequency', '4',
    '--learning_rate', '1e-4',
    '--weight_decay', '1e-4',
    '--lr_decay_rate', '0.1',
    '--val_n_clips_per_sample', '3',
    '--temp', '1.0',
    '--aug',
    '--epochs', '25',
    '--trial', '0',
    '--model_path', MODEL_PATH,
    '--comet_api_key', COMET_API_KEY
]

save_file_best = train_first_stage(stage_1_args)

## Stage 2

In [ ]:
stage_2_args = [
    '--data_folder', DATA_DIR,
    '--project_name', 'coreeecho-training-stage-2',
    '--model', 'uniformer_small',
    '--num_workers', '8',
    '--batch_size', '16',
    '--frames', '36',
    '--frequency', '4',
    '--learning_rate', '1e-4',
    '--weight_decay', '1e-4',
    '--val_n_clips_per_sample', '3',
    '--aug',
    '--epochs', '4',
    '--trial', '0',
    '--model_path', MODEL_PATH,
    '--comet_api_key', COMET_API_KEY,
    '--pretrained_weights', save_file_best
]

save_file_best = train_second_stage(stage_2_args)

# Evaluation